### Import Packages

In [2]:
# Ensure to install the following packages:
# pandas, geopandas, matplotlib

#Importing the required libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

### Load Datasets

In [ ]:
# Replace 'health_facilities.csv' with the path to your dataset
health_facilities_df = pd.read_csv('health_facilities.csv')
# Assuming 'latitude' and 'longitude' columns exist
gdf_health_facilities = gpd.GeoDataFrame(
    health_facilities_df, 
    geometry=gpd.points_from_xy(health_facilities_df.longitude, health_facilities_df.latitude),
    crs="EPSG:4326"  # Make sure to use the correct coordinate reference system (CRS)
)
